In [7]:
# para dados randomicos
import random 
from datetime import datetime, timedelta
import sqlite3 
import pandas as pd

def generate_sales_data(num_records=500):
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    products = list(range(101, 111))  # 10 produtos
    regions = ['Norte', 'Sul', 'Leste', 'Oeste', 'Centro']
    
    sales_data = []
    for i in range(1, num_records + 1):
        sale_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
        product_id = random.choice(products)
        customer_id = random.randint(1001, 2000)
        quantity = random.randint(1, 10)
        unit_price = round(random.uniform(10.0, 100.0), 2)
        total_value = round(quantity * unit_price, 2)
        seller_id = random.randint(1, 20)
        region = random.choice(regions)
        
        sale = {
            "ID": i,
            "Data_Venda": sale_date.strftime('%Y-%m-%d'),
            "ID_Produto": product_id,
            "ID_Cliente": customer_id,
            "Quantidade": quantity,
            "Valor_Unitario": unit_price,
            "Valor_Total": total_value,
            "ID_Vendedor": seller_id,
            "Regiao": region
        }
        sales_data.append(sale)
    
    return sales_data

# Gerar os dados acima 
sales_data = generate_sales_data(500)

# Criar Df Dataframe
df = pd.DataFrame(sales_data)

# Conectar ao banco de dados SQLite
conn = sqlite3.connect("vendas.db")  # Cria o arquivo vendas.db no diretório atual
cursor = conn.cursor()

# Criar a tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS vendas (
    ID INTEGER PRIMARY KEY,
    Data_Venda DATE,
    ID_Produto INTEGER,
    ID_Cliente INTEGER,
    Quantidade INTEGER,
    Valor_Unitario DECIMAL(10,2),
    Valor_Total DECIMAL(10,2),
    ID_Vendedor INTEGER,
    Regiao VARCHAR(50)
)
""")

# Inserir os dados em DB
df.to_sql("vendas", conn, if_exists="replace", index=False)

# Confirmar mudanças salvando com commit e close na conexão
conn.commit()
conn.close()

print("Banco de dados SQLite criado!")


Banco de dados SQLite criado com sucesso!


In [8]:
# Reabrir a conexão para consulta
conn = sqlite3.connect("vendas.db")
cursor = conn.cursor()

# Executar uma consulta SQL
cursor.execute("SELECT * FROM vendas LIMIT 10")
rows = cursor.fetchall()

# Exibir os dados
for row in rows:
    print(row)

# Fechar a conexão
conn.close()


(1, '2023-05-19', 102, 1666, 10, 56.63, 566.3, 8, 'Norte')
(2, '2023-10-17', 110, 1843, 10, 24.32, 243.2, 11, 'Centro')
(3, '2023-12-27', 110, 1688, 10, 74.41, 744.1, 5, 'Oeste')
(4, '2023-02-07', 104, 1706, 10, 29.94, 299.4, 7, 'Oeste')
(5, '2023-09-01', 103, 1027, 8, 52.61, 420.88, 14, 'Norte')
(6, '2023-01-06', 101, 1896, 9, 76.42, 687.78, 6, 'Leste')
(7, '2023-02-27', 110, 1663, 3, 84.28, 252.84, 3, 'Leste')
(8, '2023-01-15', 108, 1956, 8, 78.78, 630.24, 14, 'Centro')
(9, '2023-02-07', 104, 1563, 10, 44.02, 440.2, 17, 'Norte')
(10, '2023-12-05', 108, 1264, 1, 22.42, 22.42, 12, 'Oeste')


In [26]:
# mostrando os 10 primeiros registros
print(df.head(10))

In [12]:
# Total das vendas por dia
df_vendas_diarias = df.groupby('Data_Venda').agg({'Valor_Total': 'sum'}).reset_index()

print(df_vendas_diarias.head(5))

   Data_Venda  Valor_Total
0  2023-01-02       184.10
1  2023-01-03      1468.27
2  2023-01-04      1160.91
3  2023-01-05       463.10
4  2023-01-06      1162.18


In [13]:
# distinct, caso houver
df = df.drop_duplicates()

In [19]:


# DAtaLake
#biblioteca importada
import os

# definir o diretorio 
path_base = 'data_lake/s3_bucket/'

# Salvar os dados em partições por ano/mês/dia
for _, row in df_vendas_diarias.iterrows():
    ano, mes, dia = row['Data_Venda'].split('-')
    diretorio = os.path.join(path_base, ano, mes, dia)
    os.makedirs(diretorio, exist_ok=True)
    arquivo_caminho = os.path.join(diretorio, f'vendas_{ano}{mes}{dia}.csv')
    
    # salvar o dataframe em .csv
    row.to_frame().T.to_csv(arquivo_caminho, index=False)




In [25]:
# Convertendo para o formato datetime
df['Data_Venda'] = pd.to_datetime(df['Data_Venda'])

# Extract de ano e mês
df['ano'] = df['Data_Venda'].dt.year
df['mes'] = df['Data_Venda'].dt.month

# Agrupando por ano e mês e somando
df_agrupado = df.groupby(['ano', 'mes'])['Valor_Total'].sum().reset_index()

# Renomeando as colunas 
df_agrupado = df_agrupado.rename(columns={"Valor_Total": "total_de_vendas"})

# SortBy nos resultados por ano e mês
df_agrupado = df_agrupado.sort_values(by=['ano', 'mes'])

# printando o resultado do DF = dataFrame
print(df_agrupado)

     ano  mes  total_de_vendas
0   2023    1         15105.03
1   2023    2         11332.06
2   2023    3         12670.19
3   2023    4          9948.33
4   2023    5         14165.05
5   2023    6         14996.30
6   2023    7         11241.94
7   2023    8         15025.80
8   2023    9          9574.32
9   2023   10         13055.89
10  2023   11         14915.45
11  2023   12         15098.35
